<a href="https://colab.research.google.com/github/ajgquional/LiL_Learning-TinyML/blob/main/tflite_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization Challenge

In this challenge, you will have to quantize a trained model and check for changes in model metrics. You will have to quantize your model into both INT8 and FLOAT16 and then compare the drop in accuracy and reduction in model size.

There are cells to train a model on the MNIST dataset. There is also a function that you can use to calculate the accuracy of the model.

You will have to finish the cells with TODO. You can expand the cells in the solution notebook if you get stuck or to verify your answer.

## Importing Packages

The first step is to import the packages we need. We will be using `tensorflow` and `keras` to train our models. We will also be using `numpy` to evaluate the performance of our model. Later on, we will be using `os` to find the size of our original and quantized models.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Training a model on MNIST
TensorFlow provides an easy API to download the MNIST dataset and separate it into training and testing dataset. So first we will use that to load our data.

We will preprocess our data by dividing all the pixel values by 255 to normalize them.

Next we will create a simple CNN model with two convolutional layers and two dense layers.

Finally, we can compile and train our model.


In [2]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Creating the model
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=24, kernel_size=(3, 3), activation='relu'),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(10)
])

# Training the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=2,
  validation_split=0.1,
)

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/2
1688/1688 [==============================] - 62s 36ms/step - loss: 0.1856 - accuracy: 0.9431 - val_loss: 0.0646 - val_accuracy: 0.9820
Epoch 2/2
1688/1688 [==============================] - 61s 36ms/step - loss: 0.0587 - accuracy: 0.9827 - val_loss: 0.0590 - val_accuracy: 0.9855


## Evaluation Model Performance
Below is a function that take a tflite interpreter and runs inference on the test set of the MNIST data. We can use this to evaluate the performance of our original and quantized models

In [3]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

## TODO: Quantize the model weights to INT8

In the cell below, write code that take the keras `model` we trained before and then performs weight quanization to INT8.

Then create a `tflite` interpreter and use the `evaluate_model()` function to check the accuracy of the model

In [ ]:
#TODO Perform Weight Quantization

In [ ]:
#TODO Evaluate Model Performance

In [4]:
# @title Solution: Quantize the model weights to INT8
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

int8_quantized_tflite_model = converter.convert()

In [5]:
# @title Solution: Evaluate the model performance
interpreter = tf.lite.Interpreter(model_content=int8_quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('INT8 Quantized Model Test Accuracy:', test_accuracy)

# quantized model has dropped to 98.18% accuracy compared to the 98.68% accuracy of the orginal model

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


INT8 Quantized Model Test Accuracy: 0.9818


## TODO: Quantize the model weights to FLOAT16

In the cell below, write code that take the keras `model` we trained before and then performs weight quanization to FLOAT16.

Then create a `tflite` interpreter and use the `evaluate_model()` function to check the accuracy of the model

In [ ]:
#TODO Perform Weight Quantization

In [ ]:
#TODO Evaluate Model Performance

In [6]:
# @title Solution: Quantize the model weights to FLOAT16
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
float16_quantized_tflite_model = converter.convert()

In [7]:
# @title Solution: Evaluate the model performance
interpreter = tf.lite.Interpreter(model_content=float16_quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('FLOAT16 Quantized Model Test Accuracy:', test_accuracy)

# accuracy of the FLOAT16 model is 98.16%

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


FLOAT16 Quantized Model Test Accuracy: 0.9816


## TODO: Save the model files and calculate their size
In the cell below, write code to save both the original and quantized model files and then calculate the model size. You can use the `os.path.getsize()` function to get the size of a file.

In [ ]:
#TODO Calculate the model size

In [8]:
# @title Solution: Save model file and calculate their size

original_converter = tf.lite.TFLiteConverter.from_keras_model(model)
original_tflite_model = original_converter.convert()

with open('original_model', 'wb') as f:
  f.write(original_tflite_model)

with open('int8_model', 'wb') as f:
  f.write(int8_quantized_tflite_model)

with open('float16_model', 'wb') as f:
  f.write(float16_quantized_tflite_model)

print("Original model size in Mb:", os.path.getsize('original_model') / float(2**20))
print("INT8 model size in Mb:", os.path.getsize('int8_model') / float(2**20))
print("FLOAT16 model size in Mb:", os.path.getsize('float16_model') / float(2**20))

Original model size in Mb: 0.8595390319824219
INT8 model size in Mb: 0.21955108642578125
FLOAT16 model size in Mb: 0.4326019287109375
